# Negative Harmonizer

We need to install mido and easydict.

In [1]:
pip install mido

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install easydict

  Stored in directory: C:\Users\palod\AppData\Local\pip\Cache\wheels\9a\88\ec\085d92753646b0eda1b7df49c7afe51a6ecc496556d3012e2e
Successfully built easydict
Note: you may need to restart the kernel to use updated packages.


Error processing line 7 of C:\Users\palod\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\palod\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


In [2]:
import argparse   #defining arguments in a user friendly way
import os         #misc interface
import mido       #midi
import easydict   #dict values as attributes

In [19]:
def get_mirror_axis(tonic): #we define the mirror axis above the tonic note (input)
    return tonic + 3.5


def mirror_note_over_axis(note, axis): #inputs a note and its mirror axis
    original_note_distance = axis - note
    return int(axis + original_note_distance)


def find_average_octave_of_tracks(mid): #we need to find the octave average in order to keep the song balanced
    octaves = {}
    for i, track in enumerate(mid.tracks):
        track_avg_notes = []
        for message in track:
            if message.type == 'note_on':
                track_avg_notes.append(message.note)
        if len(track_avg_notes) > 0:
            track_avg_note = sum(track_avg_notes) / len(track_avg_notes)
            octaves[i] = track_avg_note
            try:
                track_name = track.name
            except AttributeError:
                track_name = i
            print(track_name, track_avg_note)
    return octaves


def mirror_all_notes(mid, mirror_axis, ignored_channels): #mirror the notes that are not to be ignored
    for track in mid.tracks:
        for message in track:
            if message.type == 'note_on' or message.type == 'note_off':
                if message.channel not in ignored_channels:
                    mirrored_note = mirror_note_over_axis(message.note, mirror_axis)
                    message.note = mirrored_note
    return


def transpose_back_to_original_octaves(mid, original_octaves, new_octaves, ignored_channels):
    for i, track in enumerate(mid.tracks):
        if i in original_octaves:
            notes_distance = original_octaves[i] - new_octaves[i]
            octaves_to_transpose = round(notes_distance / 12)
            for message in track:
                if message.type == 'note_on' or message.type == 'note_off':
                    if message.channel not in ignored_channels:
                        transposed_note = message.note + (octaves_to_transpose * 12)
                        message.note = int(transposed_note)


def invert_tonality(mid, tonic, ignored_channels, adjust_octaves):
    mirror_axis = get_mirror_axis(tonic)

    if adjust_octaves:
        print("---")
        print("original average note values:")
        original_octaves = find_average_octave_of_tracks(mid)

    mirror_all_notes(mid, mirror_axis, ignored_channels)

    if adjust_octaves:
        print("---")
        print("new average note values:")
        new_octaves = find_average_octave_of_tracks(mid)

        transpose_back_to_original_octaves(mid, original_octaves, new_octaves, ignored_channels)

        print("---")
        print("adjusted average note values:")
        find_average_octave_of_tracks(mid)
    return


def main(input_file, tonic, ignored_channels, adjust_octaves):
    root, ext = os.path.splitext(input_file)
    mid = mido.MidiFile(input_file)

    invert_tonality(mid, tonic, ignored_channels, adjust_octaves)
    mid.save(root + "_negative" + ext)

In [20]:
if __name__ == '__main__':    
    args = easydict.EasyDict({
        "file": 'jingle_bells.mid',
        "tonic": 60, #default: 60 --> C5 
        "ignore": [], #if we want to ignore the drums track, it is usually 9
        "adjust_octaves": False #transpose octaves to keep bass instruments low
    })

    main(input_file=args.file, tonic=args.tonic, ignored_channels=args.ignore, adjust_octaves=args.adjust_octaves)